In [1]:
import pandas as pd
import numpy as np

from nltk.stem import PorterStemmer

import torch
from transformers import AutoTokenizer, AutoModel
import plotly.express as px
import datetime

# Генерация эмбеддингов для строк

In [2]:
print(datetime.datetime.now())

2022-03-09 01:43:53.519274


## Загрузка данных

In [3]:
df_education = pd.read_csv('data/uuid_x_education_fields.csv', index_col=0)

## LaBSE

https://huggingface.co/cointegrated/LaBSE-en-ru

In [4]:
def get_education_enbeddings(df_education):
    
    tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
    model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

    embedding_vec = []
    df_education_study = df_education[['education_field_of_study', 'education_school_name']].drop_duplicates().reset_index(drop=True)

    for i, row in enumerate(df_education_study['education_field_of_study']):
        
        print('{} / {}'.format(i+1, df_education_study.shape[0]), end="\r")

        embeddings = []

        if row == row:
            encoded_input = tokenizer(row, padding=True, truncation=True, max_length=64, return_tensors='pt')

            with torch.no_grad():
                model_output = model(**encoded_input)

            embeddings = model_output.pooler_output
            embeddings = torch.nn.functional.normalize(embeddings)

        embedding_vec.extend(np.array(embeddings))

    df_embeddings = pd.DataFrame(embedding_vec)
    
    df_embeddings = pd.concat(
        [
            df_embeddings, 
            df_education_study
        ], axis=1
    )
    
    return df_embeddings

In [10]:
df_embeddings = pd.read_csv('data/embeddings/education_field_of_study.csv', index_col=0)
vec_embedding = df_embeddings.loc[:, ~df_embeddings.columns.isin(
    ['education_field_of_study', 'education_school_name']
)]

In [11]:
vec_embedding

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.042175,-0.033193,0.005290,-0.016716,-0.051060,0.017075,-0.023448,-0.016728,0.003121,-0.004781,...,0.017823,0.032731,-0.006918,-0.037925,0.051170,0.010998,-0.008977,-0.045202,0.014439,-0.063344
1,-0.071434,0.036790,-0.000054,-0.030793,-0.013560,-0.012521,-0.018972,-0.031263,-0.077431,0.044558,...,-0.044307,-0.027909,-0.017733,0.026550,0.047359,0.013292,-0.009682,0.000672,0.002206,-0.052806
2,-0.001919,-0.070319,-0.062744,-0.034841,-0.055945,0.019722,0.000266,-0.008627,-0.022688,-0.045811,...,-0.024135,0.041871,-0.004742,-0.050070,0.033812,-0.017038,0.017684,-0.022927,0.000963,-0.045907
3,0.021180,-0.020406,-0.045546,-0.031703,-0.010687,-0.033445,0.049052,0.044700,-0.044263,-0.038632,...,0.049012,-0.011741,0.052005,0.024124,0.001285,0.005709,-0.019484,-0.027764,0.048508,-0.035169
4,-0.019024,-0.023689,0.010003,0.049358,0.026568,-0.031377,-0.033795,-0.012664,-0.027234,0.008116,...,0.037055,0.002251,0.041033,-0.006926,-0.043469,-0.043901,-0.018180,-0.024506,-0.006130,0.023132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9845,0.018670,-0.072588,-0.009903,-0.013927,-0.049256,0.016558,-0.006821,0.022603,0.023021,-0.014726,...,-0.009755,-0.009294,0.024573,-0.069479,0.055006,-0.009563,-0.031845,-0.038309,0.024864,-0.003429
9846,0.039360,-0.022668,0.034627,-0.026534,-0.054166,-0.010556,-0.003256,-0.041811,-0.036811,-0.044241,...,0.029000,0.048901,-0.004262,-0.059563,0.053967,0.013228,0.035234,-0.048790,-0.032712,-0.027862
9847,-0.028967,-0.050401,0.012432,0.032659,-0.068817,-0.000040,-0.036729,-0.008264,-0.052668,-0.008228,...,-0.002301,-0.041896,0.068136,-0.036875,-0.028003,-0.021308,-0.045539,-0.022693,-0.061876,-0.003989
9848,-0.021835,-0.054338,-0.050735,0.007189,-0.067372,-0.041620,0.016745,-0.041264,0.024138,-0.032282,...,-0.059227,0.054059,-0.042500,-0.066759,-0.017733,0.032399,0.010320,-0.037641,0.029807,-0.040273
